In [ ]:
class Resnet(torch.nn.Module):
    def __init__(self,acti,c_num):
        super(Resnet, self).__init__()
        self.features = torch.nn.Sequential(
            torch.nn.Conv1d(1,16,kernel_size=7,stride=2,padding=3),
            torch.nn.MaxPool1d(3,2,1),
            Bottlrneck(16,16,64,False,acti),
            Bottlrneck(64,16,64,False,acti),
            Bottlrneck(64,16,64,False,acti),
        )
        self.layers=nn.ModuleList([])
        input_channel=64
        output_channel=128
        med_channel=64
        for i in range(1,c_num):
            self.layers.append(Bottlrneck(input_channel,med_channel,output_channel,(1==i)|(4==i)|(7==i),acti))
            if (1==i)|(4==i)|(7==i):
                input_channel=input_channel*2
            elif (3==i)|(6==i):
                med_channel=med_channel*2
                output_channel=output_channel*2
        self.layers.append(torch.nn.AdaptiveAvgPool1d(1))
        self.classifer = torch.nn.Sequential(
            torch.nn.Linear(2048,1)
        )



In [13]:
c_num=3
for i in range(1,c_num+1):
    print(i)
    if (i!=c_num):
        print("a")

1
a
2
a
3


In [ ]:
class Resnet(torch.nn.Module):
    def __init__(self,acti,c_num):
        super(Resnet, self).__init__()
        self.features = torch.nn.Sequential(
            torch.nn.Conv1d(1,64,kernel_size=7,stride=2,padding=3),
            torch.nn.MaxPool1d(3,2,1),
            Bottlrneck(64,64,256,False,acti),
            Bottlrneck(256,64,256,False,acti),
            Bottlrneck(256,64,256,False,acti),
        )
        self.layers=nn.ModuleList([])
        input_channel=256
        output_channel=512
        med_channel=128
        for i in range(1,c_num):
            self.layers.append(Bottlrneck(input_channel,med_channel,output_channel,(1==i)|(4==i)|(7==i),acti))
            if (1==i)|(4==i)|(7==i):
                input_channel=input_channel*2
            elif (3==i)|(6==i):
                med_channel=med_channel*2
                output_channel=output_channel*2
        self.layers.append(torch.nn.AdaptiveAvgPool1d(1))
        self.classifer = torch.nn.Sequential(
            torch.nn.Linear(2048,1)
        )

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections.abc import Iterable

class Bottlrneck(torch.nn.Module):
    def __init__(self,In_channel,Med_channel,Out_channel,downsample=False):
        super(Bottlrneck, self).__init__()
        self.stride = 1
        if downsample == True:
            self.stride = 2

        self.layer = torch.nn.Sequential(
            torch.nn.Conv1d(In_channel, Med_channel, 1, self.stride),
            torch.nn.BatchNorm1d(Med_channel),
            torch.nn.ReLU(),
            torch.nn.Conv1d(Med_channel, Med_channel, 3, padding=1),
            torch.nn.BatchNorm1d(Med_channel),
            torch.nn.ReLU(),
            torch.nn.Conv1d(Med_channel, Out_channel, 1),
            torch.nn.BatchNorm1d(Out_channel),
            torch.nn.ReLU(),
        )

        if In_channel != Out_channel:
            self.res_layer = torch.nn.Conv1d(In_channel, Out_channel,1,self.stride)
        else:
            self.res_layer = None

    def forward(self,x):
        if self.res_layer is not None:
            residual = self.res_layer(x)
        else:
            residual = x
        return self.layer(x)+residual


class Resnet(torch.nn.Module):
    def __init__(self,in_channels=1,classes=1):
        super(Resnet, self).__init__()
        self.features = torch.nn.Sequential(
            torch.nn.Conv1d(in_channels,64,kernel_size=7,stride=2,padding=3),
            torch.nn.MaxPool1d(3,2,1),

            Bottlrneck(64,64,256,False),
            Bottlrneck(256,64,256,False),
            Bottlrneck(256,64,256,False),
            #
            
            Bottlrneck(256,128,512, True),
            Bottlrneck(512,128,512, False),
            Bottlrneck(512,128,512, False),
            #
            Bottlrneck(512,256,1024, True),
            Bottlrneck(1024,256,1024, False),
            Bottlrneck(1024,256,1024, False),
            #
            Bottlrneck(1024,512,2048, True),
            Bottlrneck(2048,512,2048, False),
            Bottlrneck(2048,512,2048, False),

            torch.nn.AdaptiveAvgPool1d(1)
        )
        self.classifer = torch.nn.Sequential(
            torch.nn.Linear(2048,classes)
        )

    def forward(self,x):
        x = self.features(x)
        x = x.view(-1,2048)
        x = self.classifer(x)
        return x

#纵向结构
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet,self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv1d(1, 16, kernel_size=21, padding=0),
            nn.BatchNorm1d(16),
            nn.ReLU()#探索激活函数的性能
        )
        self.conv2 = nn.Sequential(
            nn.Conv1d(16, 32, kernel_size=19, padding=0),
            nn.BatchNorm1d(32),
            nn.ReLU()
        )
        self.conv3 = nn.Sequential(
            nn.Conv1d(32, 64, kernel_size=17, padding=0),
            nn.BatchNorm1d(64),
            nn.ReLU()
        )
        self.fc = nn. Linear(38080,1) #8960 ,17920
        self.drop = nn.Dropout(0.2)

    def forward(self,out):
        out = self.conv1(out)
        out = self.conv2(out)
        out = self.conv3(out)
        out = out.view(out.size(0),-1)
        # print(out.size(1))
        out = self.fc(out)
        return out


class AlexNet(nn.Module):
    def __init__(self, num_classes=1, reduction=16):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            # conv1
            nn.Conv1d(1, 16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(num_features=16),
            nn.ReLU(inplace=True),
            # nn.LeakyReLU(inplace=True),
            nn.MaxPool1d(kernel_size=2, stride=2),
            # conv2
            nn.Conv1d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(num_features=32),
            nn.ReLU(inplace=True),
            # nn.LeakyReLU(inplace=True),
            nn.MaxPool1d(kernel_size=2, stride=2),
            # conv3
            nn.Conv1d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.LeakyReLU(inplace=True),
            nn.MaxPool1d(kernel_size=2, stride=2),
            # conv4
            nn.Conv1d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(num_features=128),
            nn.ReLU(inplace=True),
            nn.LeakyReLU(inplace=True),
            nn.MaxPool1d(kernel_size=2, stride=2),
            # conv5
            nn.Conv1d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(num_features=256),
            nn.ReLU(inplace=True),
            nn.MaxPool1d(kernel_size=2, stride=2),
            # SELayer(256, reduction),
            # nn.LeakyReLU(inplace=True),
            #nn.Conv1d(256, 512, kernel_size=3, stride=1, padding=1),
            #nn.BatchNorm1d(num_features=512),
            #nn.ReLU(inplace=True),
            #nn.MaxPool1d(kernel_size=2, stride=2),
            
            
        )
        self.reg = nn.Sequential(
            nn.Linear(33280,1000),#根据自己数据集修改
            nn.ReLU(inplace=True),
            # nn.LeakyReLU(inplace=True),
            nn.Linear(1000, 500),
            nn.ReLU(inplace=True),
            # nn.LeakyReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(500, num_classes),
        )

    def forward(self, x):
        out = self.features(x)
        out = out.flatten(start_dim=1)
        out = self.reg(out)
        return out

class Inception(nn.Module):
    def __init__(self,in_c,c1,c2,c3,out_C):
        super(Inception,self).__init__()
        self.p1 = nn.Sequential(
            nn.Conv1d(in_c, c1,kernel_size=1,padding=0),
            nn.Conv1d(c1, c1, kernel_size=3, padding=1)
        )
        self.p2 = nn.Sequential(
            nn.Conv1d(in_c, c2,kernel_size=1,padding=0),
            nn.Conv1d(c2, c2, kernel_size=5, padding=2)

        )
        self.p3 = nn.Sequential(
            nn.MaxPool1d(kernel_size=3,stride=1,padding=1),
            nn.Conv1d(in_c, c3,kernel_size=3,padding=1),
        )
        self.conv_linear = nn.Conv1d((c1+c2+c3), out_C, 1, 1, 0, bias=True)
        self.short_cut = nn.Sequential()
        if in_c != out_C:
            self.short_cut = nn.Sequential(
                nn.Conv1d(in_c, out_C, 1, 1, 0, bias=False),

            )
    def forward(self, x):
        p1 = self.p1(x)
        p2 = self.p2(x)
        p3 = self.p3(x)
        out =  torch.cat((p1,p2,p3),dim=1)
        out += self.short_cut(x)
        return out




class DeepSpectra(nn.Module):
    def __init__(self):
        super(DeepSpectra, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv1d(1, 16, kernel_size=5, stride=3, padding=0)
        )
        self.Inception = Inception(16, 32, 32, 32, 96)
        self.fc = nn.Sequential(
            nn.Linear(20640, 5000),
            nn.Dropout(0.5),
            nn.Linear(5000, 1)
        )
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.Inception(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x


class DenseLayer(torch.nn.Module):
    def __init__(self,in_channels,middle_channels=128,out_channels=32):
        super(DenseLayer, self).__init__()
        self.layer = torch.nn.Sequential(
            torch.nn.BatchNorm1d(in_channels),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv1d(in_channels,middle_channels,1),
            torch.nn.BatchNorm1d(middle_channels),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv1d(middle_channels,out_channels,3,padding=1)
        )
    def forward(self,x):
        return torch.cat([x,self.layer(x)],dim=1)


class DenseBlock(torch.nn.Sequential):
    def __init__(self,layer_num,growth_rate,in_channels,middele_channels=128):
        super(DenseBlock, self).__init__()
        for i in range(layer_num):
            layer = DenseLayer(in_channels+i*growth_rate,middele_channels,growth_rate)
            self.add_module('denselayer%d'%(i),layer)

class Transition(torch.nn.Sequential):
    def __init__(self,channels):
        super(Transition, self).__init__()
        self.add_module('norm',torch.nn.BatchNorm1d(channels))
        self.add_module('relu',torch.nn.ReLU(inplace=True))
        self.add_module('conv',torch.nn.Conv1d(channels,channels//2,3,padding=1))
        self.add_module('Avgpool',torch.nn.AvgPool1d(2))


class DenseNet(torch.nn.Module):
    def __init__(self,layer_num=(6,12,24,16),growth_rate=32,init_features=64,in_channels=1,middele_channels=128,classes=1):
        super(DenseNet, self).__init__()
        self.feature_channel_num=init_features
        self.conv=torch.nn.Conv1d(in_channels,self.feature_channel_num,7,2,3)
        self.norm=torch.nn.BatchNorm1d(self.feature_channel_num)
        self.relu=torch.nn.ReLU()
        self.maxpool=torch.nn.MaxPool1d(3,2,1)

        self.DenseBlock1=DenseBlock(layer_num[0],growth_rate,self.feature_channel_num,middele_channels)
        self.feature_channel_num=self.feature_channel_num+layer_num[0]*growth_rate
        self.Transition1=Transition(self.feature_channel_num)

        self.DenseBlock2=DenseBlock(layer_num[1],growth_rate,self.feature_channel_num//2,middele_channels)
        self.feature_channel_num=self.feature_channel_num//2+layer_num[1]*growth_rate
        self.Transition2 = Transition(self.feature_channel_num)

        self.DenseBlock3 = DenseBlock(layer_num[2],growth_rate,self.feature_channel_num//2,middele_channels)
        self.feature_channel_num=self.feature_channel_num//2+layer_num[2]*growth_rate
        self.Transition3 = Transition(self.feature_channel_num)

        self.DenseBlock4 = DenseBlock(layer_num[3],growth_rate,self.feature_channel_num//2,middele_channels)
        self.feature_channel_num=self.feature_channel_num//2+layer_num[3]*growth_rate

        self.avgpool=torch.nn.AdaptiveAvgPool1d(1)

        self.classifer = torch.nn.Sequential(
            torch.nn.Linear(self.feature_channel_num, self.feature_channel_num//2),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.5),
            torch.nn.Linear(self.feature_channel_num//2, classes),

        )


    def forward(self,x):
        x = self.conv(x)
        x = self.norm(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.DenseBlock1(x)
        x = self.Transition1(x)

        x = self.DenseBlock2(x)
        x = self.Transition2(x)

        x = self.DenseBlock3(x)
        x = self.Transition3(x)

        x = self.DenseBlock4(x)
        x = self.avgpool(x)
        x = x.view(-1,self.feature_channel_num)
        x = self.classifer(x)

        return x

    
# class DenseNet(torch.nn.Module):
#     def __init__(self,layer_num=(6,12,24,16),growth_rate=32,init_features=64,in_channels=1,middele_channels=128,classes=1):
#         super(DenseNet, self).__init__()
#         self.feature_channel_num=init_features
#         self.features = torch.nn.Sequential(
#             torch.nn.Conv1d(in_channels,self.feature_channel_num,7,2,3),
#             torch.nn.BatchNorm1d(self.feature_channel_num),
#             torch.nn.ReLU(),
#             torch.nn.MaxPool1d(3,2,1),
#         )
#         self.DenseBlock1=DenseBlock(layer_num[0],growth_rate,self.feature_channel_num,middele_channels)
#         self.feature_channel_num=self.feature_channel_num+layer_num[0]*growth_rate
#         self.Transition1=Transition(self.feature_channel_num)
#         self.layers=nn.ModuleList([])
#         for i in range(1,4):
#             self.layers.append(DenseBlock(layer_num[i],growth_rate,self.feature_channel_num//2,middele_channels))
#             self.feature_channel_num=self.feature_channel_num//2+layer_num[i]*growth_rate
#             if (i==1)|(i==2):
#                 self.layers.append(Transition(self.feature_channel_num))
#         self.layers.append(torch.nn.AdaptiveAvgPool1d(1))
#         self.classifer = torch.nn.Sequential(
#             torch.nn.Linear(self.feature_channel_num, self.feature_channel_num//2),
#             torch.nn.ReLU(),
#             torch.nn.Dropout(0.5),
#             torch.nn.Linear(self.feature_channel_num//2, classes),

#         )


#     def forward(self,x):
#         x = self.features(x)

#         x = self.DenseBlock1(x)
#         x = self.Transition1(x)
#         out = x
#         for layer in self.layers:
#             out = layer(out)    
#         out = x.view(-1,self.feature_channel_num)
#         out = self.classifer(out)

#         return out